In [1]:
import os
import pandas as pd
from colorama import Fore, Back, Style

print(f"{os.getcwd()}")
print(f"{os.listdir()}")

/home/georger/WorkDocuments/Sxoli/Ptyxiaki/Ptyxiaki-Repository/Python_Scripts/Data_Analysis
['Powerfleet_API_Management.py', 'JSON_response.json', '__pycache__', 'data_analysis.ipynb', 'response_data_set.csv', 'Powerfleet_API_CredentialsI.txt']


Convert **JSON** to **CSV** so that **pandas** be able to use it


In [2]:
import json
import pandas as pd
from Powerfleet_API_Management import PowerfleetAPIManager as powerfleet_API
from colorama import Fore, Style

startDate = "2024-10-23 15:32:24"  # Indicative start date
endDate = "2024-11-25 15:32:24"  # Indicative end date
api_manager = powerfleet_API(startDate, endDate)  # Get data

output_file = "response_data_set.csv"

# Retrieve response data from the API
response_data = api_manager.retrieve_response()

# Write the JSON response data to a file
with open('JSON_response.json', 'w') as json_file:
    json.dump(response_data, json_file, indent=4)

try:
    # Read the JSON file into a pandas DataFrame
    df = pd.read_json('JSON_response.json')

    # Add the start and end date columns to the DataFrame
    df['startDate'] = startDate
    df['endDate'] = endDate

    # Write the DataFrame to CSV
    df.to_csv(output_file, index=False)

    print(Fore.GREEN + f"CSV file saved as {output_file}")
    print(Style.RESET_ALL)

except ValueError as e:
    print(Fore.RED + "Error: The JSON file structure is not appropriate for conversion.", e)
    print(Style.RESET_ALL)


/home/georger/WorkDocuments/Sxoli/Ptyxiaki/Ptyxiaki-Repository/Python_Scripts/Data_Analysis
os.getcwd()='/home/georger/WorkDocuments/Sxoli/Ptyxiaki/Ptyxiaki-Repository/Python_Scripts/Data_Analysis'
['Powerfleet_API_Management.py', 'JSON_response.json', '__pycache__', 'data_analysis.ipynb', 'response_data_set.csv', 'Powerfleet_API_CredentialsI.txt']
Error: The file './Powerfleet_API_CredentialsI.txt,' was not found.
Error: Missing 'cid' or 'api_key' in the credentials file.

CSV file saved as response_data_set.csv

